# Binding Prediction via API

http://tools.iedb.org/main/tools-api/

## MHC I

In [13]:
import requests

In [14]:
# settings
data = {'method':'smm',
        'sequence_text':'SLYNTVATLYCVHQRIDV',
        'allele':'HLA-A*01:01',
        'length':'9'}

In [10]:
# perform prediction
site = 'http://tools-cluster-interface.iedb.org/tools_api/mhci/'  
r = requests.post(site, data=data)

In [23]:
import csv

# save into CSV format
with open('test.csv', mode='w') as write_file:
    file_writer = csv.writer(write_file, delimiter=',')
    
    for each_row in r.text.split('\n'):
        file_writer.writerow(each_row.split('\t'))

allele	seq_num	start	end	length	peptide	ic50	percentile_rank
HLA-A*01:01	1	2	10	9	LYNTVATLY	1286.92	1.3
HLA-A*01:01	1	4	12	9	NTVATLYCV	6061.08	5.5
HLA-A*01:01	1	3	11	9	YNTVATLYC	27896.52	21
HLA-A*01:01	1	8	16	9	TLYCVHQRI	36774.77	24
HLA-A*01:01	1	5	13	9	TVATLYCVH	49607.80	30
HLA-A*01:01	1	10	18	9	YCVHQRIDV	56565.29	32
HLA-A*01:01	1	7	15	9	ATLYCVHQR	83090.24	39
HLA-A*01:01	1	1	9	9	SLYNTVATL	85418.12	40
HLA-A*01:01	1	6	14	9	VATLYCVHQ	282195.45	68
HLA-A*01:01	1	9	17	9	LYCVHQRID	491529.97	79

